In [1]:
import os
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery

client = AzureOpenAI(
  api_key = os.environ["AOAI_API_KEY"],  
  api_version = os.environ["AOAI_API_VERSION"],
  azure_endpoint = os.environ["AOAI_ENDPOINT"]
)

search_client = SearchClient(
    endpoint= os.environ["SEARCH_SERVICE_ENDPOINT"],
    index_name="docs3",
    credential=AzureKeyCredential(os.environ["SEARCH_QUERY_KEY"])
)

prompt= "有給休暇について教えてください"

def generate_embeddings(prompt, model="text-embedding-3-small-deploy"): # model = "deployment_name"
    response = client.embeddings.create(input = prompt, model=model).data[0].embedding
    return response
    # print(response[:500])

# response = generate_embeddings(prompt)

vector_query = VectorizedQuery(
        vector=generate_embeddings(prompt),
        k_nearest_neighbors=3,  # 上位3件の結果を取得します
        fields="contentVector" # ベクトル検索を行うフィールドを指定します
    )

results = search_client.search(
    search_text='', # ベクトル検索のみ行うためテキストクエリは空
    vector_queries = [vector_query],
    select=['id', 'content'],
)

for result in results:
    print(result)

{'id': '36', 'content': '休というを付与する １時間単位年休付与の対象者はすべての労働者とする ２時間単位年休を取得する場合の１日の年次有給休暇に相当する時間数は以 下のとおりとする ① 所定労働時間が５ 時間を超え６ 時間以下の者６ 時間 ② 所定労働時間が６ 時間を超え７ 時間以下の者７ 時間 ③ 所定労働時間が７ 時間を超え８ 時間以下の者８ 時間 ３時間単位年休は１時間単位で付与する ４本条の時間単位年休に支払われる賃金額は所定労働時間労働した場合に支払 われる通常の賃金の１時間当たりの額に取得した時間単位年休の時間数を乗じ た額とする ５上記以外の事項については前条の年次有給休暇と同様とする 第２４条 年次有給休暇の時間単位での付与 １ 労使協定を締結すれば年に５日を限度として時間単位で年次有給休暇を与えるこ とができます労基法第３９条第４項 ２ 時間単位年休の１時間分の賃金額は①平均賃金②所定労働時間労働した場合に支 払われる通常の賃金③健康保険法第４０条第１項に定める標準報酬月額を３０分の１ に相当する額１の位は四捨五入ただし③については労働者代表との書面による協 定が必要ですをその日の所定労働時間で除した額になります①③のいずれにする かは就業規則等に定めることが必要です労基法第３９条第７項 ３ 労使協定に規定しなければならない内容は次のとおりです ① 時間単位年休の対象労働者の範囲対象となる労働者の範囲を定めます ② 時間単位年休の日数５日以内の範囲で定めます前年度からの繰越しがある場合 42  であっても当該繰越し分を含めて５日以内となります ③ 年次有給休暇１日分に相当する時間単位年休の時間数１日分の年次有給休暇に対 応する所定労働時間数を基に定めます１日の所定労働時間に１時間に満たない端数 がある場合は時間単位に切り上げて計算します ④ １時間以外の時間を単位とする場合はその時間数ただし１日の所定労働時間を 上回ることはできません ４ 時間単位年休も年次有給休暇ですので事業の正常な運営を妨げる場合は使用者によ る時季変更権が認められますただし日単位での請求を時間単位に変えることや時 間単位での請求を日単位に変えることはできません 産前産後の休業 第２５条 ６週間多胎妊娠の場合は１４週間以内に出産予定の女性労働者から請求